## Various data imports

In [1]:
import pandas as pd
import os
from DB2SalesforceAPI import DB2SalesforceAPI

In [2]:
# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"npass2021HDfJLhBGDx11xWCKlEbHQKhF",
}

## Load input data 

In [3]:
#import data
cwd = os.getcwd()
print(cwd)
filename = 'nanoHUB_Learning_Management_System_Integration_Survey-Distribution_History.csv'
f_end = filename[-3:]
qual_flag = 2 #### change this based on needs
print(f_end)

D:\nanohub_salesforce_integ\additional_imports
csv


In [4]:
if f_end == 'csv': #its a csv file
    temp = pd.read_csv(cwd+'/Data Files/'+filename)
else:
    temp = pd.read_excel(cwd+'/Data Files/'+filename)

In [5]:
temp.head(2)

,Response ID,Last Name,First Name,External Data Reference,Email,Status,End Date,Link,Link Expiration
0,NaN,Aagesen,Larry,NaN,laagesen@umich.edu,Email Not Sent Yet,NaN,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14
1,NaN,Acosta,Antonio,NaN,acojim@us.es,Email Not Sent Yet,NaN,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14


In [7]:
temp.shape

(85, 9)

## Extract the qualtrics link

In [9]:
df_sf = pd.DataFrame() 
link_name = 'LMS_Qualtrics_Survey__c'
ext_id = 'Email'
expiration = 'LMS_expiration_date__c'

if qual_flag == 1:
    df_sf[link_name] = temp['Link']
    df_sf[ext_id] = temp[ext_id]
    df_sf[expiration] = temp['Link Expiration']
elif qual_flag == 2:
    df_sf[link_name] = temp['Link']
    df_sf[ext_id] = temp[ext_id]
    df_sf[expiration] = temp['Link Expiration']    

In [10]:
display(df_sf.head(2))

,LMS_Qualtrics_Survey__c,Email,LMS_expiration_date__c
0,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,laagesen@umich.edu,2021-08-10 11:36:14
1,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,acojim@us.es,2021-08-10 11:36:14


## Pull in all contacts from SF and obtain their contact IDs

In [11]:
# create DB2 to Salesforce API object
db_pull = DB2SalesforceAPI(sf_login_params)

sf_data = db_pull.query_data('select Id,Email from Contact')

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000Ww7cMAAR
{"id":"7505w00000Ww7cMAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-06-11T17:46:30.000+0000","systemModstamp":"2021-06-11T17:46:31.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Ww7cMAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-06-11T17:46:30.000+0000","systemModstamp":"2021-06-11T17:46:41.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":248970,"retries":0,"totalProcessingTime":9433}
[Success] Bulk job completed successfully.


In [12]:
print(sf_data.shape)

(248970, 2)


In [13]:
from copy import deepcopy
df_sf2 = deepcopy(df_sf)

In [14]:
df_sf.shape

(85, 3)

In [15]:
df_sf2 = pd.merge(df_sf2,sf_data,how='left',on=['Email'])
display(df_sf2)
print(df_sf2.shape)

,LMS_Qualtrics_Survey__c,Email,LMS_expiration_date__c,Id
0,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,laagesen@umich.edu,2021-08-10 11:36:14,0035w000031VuNCAA0
1,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,acojim@us.es,2021-08-10 11:36:14,0035w000034JucMAAS
2,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,agah@vt.edu,2021-08-10 11:36:14,0035w000034JJw8AAG
3,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,anstinedylan@gmail.com,2021-08-10 11:36:14,0035w000034ISm4AAG
4,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,brandon.blackstone@unlv.edu,2021-08-10 11:36:14,0035w000031VtDtAAK
...,...,...,...,...
89,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,xuejing19850906@gmail.com,2021-08-10 11:36:14,0035w000034JO47AAG
90,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,ykent0406@gmail.com,2021-08-10 11:36:14,0035w000034JNEFAA4
91,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,ferry@asu.edu,2021-08-10 11:36:14,0035w000034JGKfAAO
92,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,vasileska@asu.edu,2021-08-10 11:36:14,0035w000034JKq0AAG


(94, 4)


In [16]:
df_sf2 = df_sf2.drop(columns=['Email'])

In [17]:
display(df_sf2.head(5))

,LMS_Qualtrics_Survey__c,LMS_expiration_date__c,Id
0,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14,0035w000031VuNCAA0
1,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14,0035w000034JucMAAS
2,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14,0035w000034JJw8AAG
3,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14,0035w000034ISm4AAG
4,https://purdue.ca1.qualtrics.com/jfe/form/SV_a...,2021-08-10 11:36:14,0035w000031VtDtAAK


## Send to Salesforce

In [18]:
# external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
# db_s.external_id = external_id
db_s.external_id = 'Id'

Obtained Salesforce access token ...... True


In [19]:
db_s.send_data(df_sf2)

[Success] Bulk job creation successful. Job ID = 7505w00000Ww7egAAB
[Success] CSV upload successful. Job ID = 7505w00000Ww7egAAB
[Success] Closing job successful. Job ID = 7505w00000Ww7egAAB


In [20]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Ww7egAAB',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-06-11T17:47:06.000+0000',
 'systemModstamp': '2021-06-11T17:47:07.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [21]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
